In [1]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.model_selection import KFold

# Define train and test paths
train_path = 'C:/Users/Nayan/Desktop/Project/DevanagariHandwrittenDigitDataset/Train'
test_path = 'C:/Users/Nayan/Desktop/Project/DevanagariHandwrittenDigitDataset/Test'

# Define the number of classes
num_classes = 10

# Define image size and number of channels
img_rows, img_cols, img_channels = 32, 32, 1

# Define a function to load the data from the specified path
import re

def load_data(path):
    data = []
    labels = []
    pattern = r"\d+" # regular expression to match digits in folder name
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (img_rows, img_cols))
            data.append(image)
            match = re.search(pattern, folder)
            labels.append(int(match.group())) # convert matched digits to integer label
    data = np.array(data)
    data = data.reshape(data.shape[0], img_rows, img_cols, img_channels)
    data = data.astype('float32')
    data /= 255
    labels = np.array(labels)
    labels = np_utils.to_categorical(labels, num_classes)
    return data, labels

# Load the data
X, y = load_data(train_path)

# Define the CNN-LSTM model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, img_channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
from keras.layers import Reshape
from keras.layers import TimeDistributed, LSTM, Dense
model.add(Flatten())
model.add(Reshape((8, 8, 64)))
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(128)))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

In [4]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the number of folds
K = 3

# Define the KFold object
kf = KFold(n_splits=K, shuffle=True)

# Define lists to store the accuracy and loss for each fold
acc_per_fold = []
loss_per_fold = []

In [5]:
# Loop over the folds
fold_no = 1
for train_idx, val_idx in kf.split(X):
    # Split the data into training and validation sets for this fold
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Train the model on the training data for this fold
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64)
    
    # Evaluate the model on the validation data for this fold
    scores = model.evaluate(X_val, y_val, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    # Increment the fold number
    fold_no += 1

Epoch 1/20
178/178 [==============================] - 19s 82ms/step - loss: 0.0030 - accuracy: 0.9991 - val_loss: 0.0026 - val_accuracy: 0.9991
Epoch 2/20
178/178 [==============================] - 15s 83ms/step - loss: 0.0028 - accuracy: 0.9989 - val_loss: 0.0012 - val_accuracy: 0.9994
Epoch 3/20
178/178 [==============================] - 14s 76ms/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.0027 - val_accuracy: 0.9991
Epoch 4/20
178/178 [==============================] - 12s 69ms/step - loss: 0.0048 - accuracy: 0.9988 - val_loss: 0.0012 - val_accuracy: 0.9994
Epoch 5/20
178/178 [==============================] - 12s 67ms/step - loss: 6.4609e-04 - accuracy: 0.9998 - val_loss: 5.4919e-05 - val_accuracy: 1.0000
Epoch 6/20
178/178 [==============================] - 12s 68ms/step - loss: 0.0028 - accuracy: 0.9992 - val_loss: 0.0130 - val_accuracy: 0.9962
Epoch 7/20
178/178 [==============================] - 13s 71ms/step - loss: 0.0034 - accuracy: 0.9987 - val_loss: 0.0016 - val_a

In [6]:
# Print the overall accuracy and loss across all folds
print('------------------------------------------------------------------------')
print('Score per fold:')
for i in range(K):
    print(f'Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+/- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
model.save("conv_model_Final.hdf5")

------------------------------------------------------------------------
Score per fold:
Fold 1 - Loss: 0.0020910457242280245 - Accuracy: 99.982351064682%
Fold 2 - Loss: 0.000623264058958739 - Accuracy: 99.982351064682%
Fold 3 - Loss: 2.3900165615486912e-05 - Accuracy: 100.0%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 99.98823404312134 (+/- 0.008319787896050482)
> Loss: 0.0009127366496007502
